In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from fiona.crs import from_epsg

In [2]:
# import hexagons geojson file
gdf_hexagons = gpd.read_file('../hexagons.geojson')

In [3]:
gdf_hexagons.shape

(79, 4)

In [4]:
gdf_hexagons.head()

,index,x,y,geometry
0,8830e1ca31fffff,127.046889,37.500817,"POLYGON ((127.05109 37.49886, 127.05058 37.503..."
1,8830e1cae9fffff,127.061157,37.514724,"POLYGON ((127.06536 37.51277, 127.06485 37.517..."
2,8830e1ca23fffff,127.035810,37.493646,"POLYGON ((127.04001 37.49169, 127.03950 37.496..."
3,8830e1c853fffff,127.072123,37.489082,"POLYGON ((127.07633 37.48713, 127.07582 37.491..."
4,8830e1c81bfffff,127.084741,37.483211,"POLYGON ((127.08895 37.48125, 127.08844 37.485..."


In [5]:
# import residential buildings geojson file
gdf_residentials = gpd.read_file('../gangnamGu-building-residential.geojson')

In [6]:
gdf_residentials.head()

,USABILITY,HEIGHT,PNU,GRND_FLR,UGRND_FLR,DONG_NM,UFID,BC_RAT,VC_RAT,COL_ADM_SECT_CD,BLDG_PNU,OBJECTID,BD_MGT_SN,count_bus_stop,geometry
0,01000,7.90,1168011400106410010,2,1,None,1987207556994435369900000000,49.21,None,None,None,None,1168011400106410010001998,9,"MULTIPOLYGON (((127.08634 37.49399, 127.08628 ..."
1,02000,13.16,1168010800101570024,4,1,None,2003202386534451790800000000,54.62,None,None,None,None,1168010800101570024008663,2,"MULTIPOLYGON (((127.02788 37.50883, 127.02781 ..."
2,02000,0.00,1168010500100790000,12,1,6,1981205226284463626200000000,0.00,None,None,None,None,1168010500100790000014655,5,"MULTIPOLYGON (((127.06025 37.51966, 127.06031 ..."
3,01000,9.80,1168010400100110005,3,0,´Ù°¡±¸¿ë´Üµ¶ÁÖÅÃ(9°¡±¸),1995204058644466407200000000,59.76,None,None,None,None,1168010400100110005018473,10,"MULTIPOLYGON (((127.04673 37.52192, 127.04665 ..."
4,01000,0.00,1168010500100700004,2,1,None,1978204624064458510800000000,0.00,None,None,None,None,1168010500100700004015465,6,"MULTIPOLYGON (((127.05317 37.51489, 127.05315 ..."


In [7]:
def count_buildings(polygon):
    '''
    This function is for making a list of buildings that is in a hexagon area.
    -----
    input:
    polygon (Shapely.geometry.Polygon): a hexagon polygon 
    -----
    output:
    precise_matches_list (list): a list of residential buildings
    
    '''
    try:
        buildings_sindex = gdf_residentials.sindex
        possible_matches_index = list(buildings_sindex.intersection(polygon.bounds))
        possible_matches = gdf_residentials.iloc[possible_matches_index]
        # extract the list of buildings that is in a hexagon
        precise_matches_list = possible_matches[possible_matches.intersects(polygon)]['UFID'].tolist()
    except:
        # if there is no buildings that matched within a hexagon, return empty list
        precise_matches_list = []
    return precise_matches_list

In [8]:
# get list of residential buildings in each hexagon
gdf_hexagons['residentials'] = gdf_hexagons['geometry'].apply(lambda x: count_buildings(x))

In [9]:
# count the number of residential buildings in each hexagon
gdf_hexagons['count_residentials'] = gdf_hexagons['residentials'].apply(lambda x: len(x))

In [10]:
# delete the hexagon which doesn't contains any residential buildings
gdf_hexagons = gdf_hexagons.loc[gdf_hexagons['count_residentials']>10]

In [11]:
gdf_hexagons = gdf_hexagons.drop(['residentials', 'count_residentials'], axis=1)

In [12]:
gdf_hexagons.to_file('../hexagons_filtered.geojson', encoding='utf8', driver='GeoJSON')